# 手把手学习seq2seq（上）：基于循环神经网络实现








序列到序列（sequence to sequence，seq2seq）类的学习任务

# 编码器-解码器架构

## 机器翻译

机器翻译指的是将文本序列从一种语言自动翻译成另一种语言，
是典型的*序列到序列*（sequence to sequence，seq2seq）类的学习任务。

早期*统计机器翻译*（statisticalmachine translation）在这一领域一直占据主导地位，
现代机器翻译广泛使用端到端的*神经机器翻译*（neuralmachine translation）。

### BLEU

BLEU（bilingual evaluation understudy, `Papineni.Roukos.Ward.ea.2002`）被广泛用于评估输出序列的质量，
它通过测量预测序列和标签序列之间的$n$元语法的匹配度来评估预测。

BLEU定义为：

$$ \exp\left(\min\left(0, 1 - \frac{\mathrm{len}_{\text{label}}}{\mathrm{len}_{\text{pred}}}\right)\right) \prod_{n=1}^k p_n^{1/2^n},$$

其中$\mathrm{len}_{\text{label}}$表示标签序列中的词元数和
$\mathrm{len}_{\text{pred}}$表示预测序列中的词元数。
$k$是用于匹配的最长的$n$元语法，$p_n$表示$n$元语法的精确度。

**示例** 给定标签序列$A$、$B$、$C$、$D$、$E$、$F$
和预测序列$A$、$B$、$B$、$C$、$D$，
$p_1 = 4/5$、$p_2 = 3/4$、$p_3 = 1/3$和$p_4 = 0$。

* 1-gram, $A$ (match)、$B$ (match)、$B$ (match)、$C$ (match)、$D$ (match)
* 2-gram, $AB$ (match)、$BB$ (miss)、$BC$ (match)、$CD$ (match)
* 3-gram, $ABB$ (miss)、$BBC$ (miss)、$BCD$ (match)
* 4-gram, $ABBC$ (miss)、$BBCD$ (miss)


**物理意义**

* 当预测序列与标签序列完全相同时，BLEU为$1$。
* $n$元语法越长则匹配难度越大，
  所以BLEU为更长的$n$元语法的精确度分配更大的权重。
  * 具体来说，当$p_n$固定时，$p_n^{1/2^n}$
    会随着$n$的增长而增加。
* 由于预测的序列越短获得的$p_n$值越高，
  所以公式中乘法项之前的系数用于惩罚较短的预测序列。
  * 例如，当$k=2$时，给定标签序列$A$、$B$、$C$、$D$、$E$、$F$
    和预测序列$A$、$B$，尽管$p_1 = p_2 = 1$，
    惩罚因子$\exp(1-6/2) \approx 0.14$会降低BLEU。

## 编码器-解码器架构

*编码器-解码器*（encoder-decoder）架构包含了两个主要组件，即*编码器*（encoder）和*解码器*（decoder）。

<img src="img/encoder-decoder.svg" width="50%" height="50%" align="left"/>

* “编码器－解码器”架构可以将长度可变的序列作为输入和输出，因此适用于机器翻译等序列转换问题。
* 编码器将长度可变的序列作为输入，并将其转换为具有固定形状的编码状态。
* 解码器将具有固定形状的编码状态映射为长度可变的序列。

## 基于循环神经网络的实现

遵循编码器－解码器架构的设计原则，
使用两个独立的循环神经网络来设计一个序列到序列学习的模型。

编码器将长度可变的序列*编码*到循环神经网络编码器的隐状态（即输入序列的编码信息）中，
解码器基于输入序列的编码信息和输出序列已经看见的或者生成的词元来预测下一个词元。

<img src="img/seq2seq.svg" width="50%" height="50%" align="left"/>

### Encoder

假设输入序列是$x_1, \ldots, x_T$，
其中$x_t$是输入文本序列中的第$t$个词元。

当前步$t$的隐状态
$\mathbf{h}_t = f(\mathbf{x}_t, \mathbf{h}_{t-1})$
编码器通过选定的函数$q$。
将所有时间步的隐状态转换为上下文变量
$\mathbf{c} =  q(\mathbf{h}_1, \ldots, \mathbf{h}_T)$

* 最简单的做法，
  将输入序列在最后时间步的隐状态$\mathbf{h}_T$（即 $q(\mathbf{h}_1, \ldots, \mathbf{h}_T) = \mathbf{h}_T$）作为输入序列的上下文变量。

### Decoder

解码器输出$y_{t'}$的概率取决于先前的输出子序列
$y_1, \ldots, y_{t'-1}$和编码器的上下文变量$\mathbf{c}$，
即$P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$。

使用另一个循环神经网络建模解码器。
在输出序列上的任意时间步$t^\prime$，
循环神经网络将来自上一时间步的输出$y_{t^\prime-1}$
和上下文变量$\mathbf{c}$作为其输入，
然后在当前时间步将它们和上一隐状态
$\mathbf{s}_{t^\prime-1}$转换为
隐状态$\mathbf{s}_{t^\prime}$。
因此，可以使用函数$g$来表示解码器的隐藏层的变换：

$$\mathbf{s}_{t^\prime} = g(y_{t^\prime-1}, \mathbf{c}, \mathbf{s}_{t^\prime-1}).$$

在获得解码器的隐状态之后，
使用输出层和softmax操作
来计算在时间步$t^\prime$时输出$y_{t^\prime}$的条件概率分布
$P(y_{t^\prime} \mid y_1, \ldots, y_{t^\prime-1}, \mathbf{c})$。

采用多层循环神经网络实现Encoder-Decoder模型

<img src="img/seq2seq-details.svg" width="35%" height="35%" align="left"/>

### 训练

* 在机器翻译中，我们更喜欢单词级词元化（最先进的模型可能使用更高级的词元化技术）。
* mask loss：计算损失时需要考虑剔除填充值。
* teach force：在“编码器－解码器”训练中，通常采用强制*教学方法*将原始输出序列（而非预测结果）输入解码器。

### 预测：生成目标序列

采用*串行*生成目标序列。

* 最简单方式：从开始词元（“&lt;bos&gt;”）开始，直至遇到结束词元（“&lt;eos&gt;”）或者超过指定长度时，预测结束。

* *束搜索*（beam-search）生成目标序列。


#### 束搜索